# Decadal Calendar Fix

In [1]:
import xarray as xr
from rooki import rooki

In [2]:
ds_id = "c3s-cmip6-decadal.DCPP.EC-Earth-Consortium.EC-Earth3.dcppA-hindcast.s1976-r3i1p1f1.Amon.psl.gr.v20201215"


## ds with mixed gregorian and proleptic gregorian



In [22]:
result = rooki.subset(ds_id, time="1984-11/1986-10", original_files=True)

result.ok

True

In [23]:
for ds in result.datasets():
    print(ds.time.encoding.get("calendar"))

gregorian
proleptic_gregorian


In [3]:
ds_path_g = "/var/folders/qb/mg0csz190wd4rxybhhnwjln80000gn/T/metalink__vun7bgo/psl_Amon_EC-Earth3_dcppA-hindcast_s1976-r3i1p1f1_gr_198411-198510.nc"

ds_path_pg = "/var/folders/qb/mg0csz190wd4rxybhhnwjln80000gn/T/metalink__vun7bgo/psl_Amon_EC-Earth3_dcppA-hindcast_s1976-r3i1p1f1_gr_198511-198610.nc"



In [4]:
ds_p = xr.open_dataset(ds_path_g, use_cftime=True)
ds_p

<xarray.Dataset>
Dimensions:    (time: 12, bnds: 2, lat: 256, lon: 512)
Coordinates:
  * time       (time) object 1984-11-16 00:00:00 ... 1985-10-16 12:00:00
  * lat        (lat) float64 -89.46 -88.77 -88.07 -87.37 ... 88.07 88.77 89.46
  * lon        (lon) float64 0.0 0.7031 1.406 2.109 ... 357.2 357.9 358.6 359.3
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    psl        (time, lat, lon) float32 ...
Attributes: (12/48)
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               DCPP
    branch_method:             no parent
    branch_time:               0.0
    branch_time_in_child:      0.0
    branch_time_in_parent:     0.0
    ...                        ...
    cmor_version:              3.4.0
    variant_info:              Atmosphere initialization based on full-fields...
    sub_experiment_id:         s1976
    history:                   Wed Sep 25 11:00:49 2019: ncatted -O -a sub_ex...
    nominal_resolution:        100 km
    sub_experiment:            initialized near end of year 1976

In [5]:
ds_p.time.encoding.get("calendar")

'gregorian'

In [6]:
ds_pg = xr.open_dataset(ds_path_pg, use_cftime=True)
ds_pg

<xarray.Dataset>
Dimensions:    (time: 12, bnds: 2, lat: 256, lon: 512)
Coordinates:
  * time       (time) object 1985-11-16 00:00:00 ... 1986-10-16 12:00:00
  * lat        (lat) float64 -89.46 -88.77 -88.07 -87.37 ... 88.07 88.77 89.46
  * lon        (lon) float64 0.0 0.7031 1.406 2.109 ... 357.2 357.9 358.6 359.3
Dimensions without coordinates: bnds
Data variables:
    time_bnds  (time, bnds) object ...
    lat_bnds   (lat, bnds) float64 ...
    lon_bnds   (lon, bnds) float64 ...
    psl        (time, lat, lon) float32 ...
Attributes: (12/48)
    Conventions:               CF-1.7 CMIP-6.2
    activity_id:               DCPP
    branch_method:             no parent
    branch_time:               0.0
    branch_time_in_child:      0.0
    branch_time_in_parent:     0.0
    ...                        ...
    license:                   CMIP6 model data produced by EC-Earth-Consorti...
    cmor_version:              3.4.0
    sub_experiment_id:         s1976
    history:                   Wed Sep 25 11:00:52 2019: ncatted -O -a sub_ex...
    nominal_resolution:        100 km
    sub_experiment:            initialized near end of year 1976

In [7]:
ds_pg.time.encoding.get("calendar")

'proleptic_gregorian'

## apply calendar fix

In [9]:
from rook.utils import decadal_fixes

In [10]:
ds_fix = decadal_fixes.decadal_fix_calendar(None, ds_pg)
ds_fix

TypeError: can only decode Dataset or DataStore objects

In [11]:
xr.conventions.times.decode_cf_datetime?

Signature:
xr.conventions.times.decode_cf_datetime(
    num_dates,
    units: 'str',
    calendar: 'str | None' = None,
    use_cftime: 'bool | None' = None,
) -> 'np.ndarray'
Docstring:
Given an array of numeric dates in netCDF format, convert it into a
numpy array of date time objects.

For standard (Gregorian) calendars, this function uses vectorized
operations, which makes it much faster than cftime.num2date. In such a
case, the returned array will be of type np.datetime64.

Note that time unit in `units` must not be smaller than microseconds and
not larger than days.

See Also
--------
cftime.num2date
File:      ~/miniforge3/envs/rook/lib/python3.11/site-packages/xarray/coding/times.py
Type:      function

In [17]:
decoded_times = xr.conventions.times.decode_cf_datetime(
    ds_pg.time, 
    units=ds_pg.time.encoding["units"], 
    calendar="standard",
    use_cftime=True
)

TypeError: int() argument must be a string, a bytes-like object or a real number, not 'cftime._cftime.DatetimeProlepticGregorian'

In [15]:
ds_pg.time.encoding["units"]

'days since 1986-08-01 00:00:00'